In [2]:
import numpy as np
import pandas as pd

import pandas as pd
import numpy as np
import torch

import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from cleaning.pipeline import strategic_fill, shift_forecast_columns

In [3]:
project_root = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
data_path = os.path.join(project_root, 'data', 'raw_data', 'us_homestead.csv')
print(f"Trying to load from: {data_path}")
raw_data = pd.read_csv(data_path)

raw_data.rename(columns={'Date': 'time'}, inplace=True)
raw_data['time'] = pd.to_datetime(raw_data['time'], utc=True)
raw_data.set_index('time', inplace=True)
raw_data = raw_data.drop(columns=['Unnamed: 0'])


Trying to load from: /Users/aryanahri/epf_datasets/data/raw_data/us_homestead.csv


In [4]:
raw_data.head()

,Homestead_maxtempC,Homestead_mintempC,Homestead_DewPointC,Homestead_FeelsLikeC,Homestead_HeatIndexC,Homestead_WindChillC,Homestead_WindGustKmph,Homestead_cloudcover,Homestead_humidity,Homestead_precipMM,Homestead_pressure,Homestead_tempC,Homestead_visibility,Homestead_winddirDegree,Homestead_windspeedKmph,Consumption
time,,,,,,,,,,,,,,,,
2020-06-25 17:00:00+00:00,33.0,27.0,23.0,36.0,36.0,31.0,24.0,9.0,62.0,0.0,1018.0,31.0,10.0,94.0,19.0,103.0
2020-06-25 16:00:00+00:00,33.0,27.0,23.0,37.0,37.0,32.0,24.0,7.0,60.0,0.0,1018.0,32.0,10.0,96.0,20.0,99.0
2020-06-25 15:00:00+00:00,33.0,27.0,23.0,38.0,38.0,33.0,23.0,5.0,57.0,0.0,1019.0,33.0,10.0,98.0,20.0,74.0
2020-06-25 14:00:00+00:00,33.0,27.0,23.0,37.0,37.0,32.0,22.0,6.0,57.0,0.0,1019.0,32.0,10.0,96.0,19.0,82.0
2020-06-25 13:00:00+00:00,33.0,27.0,23.0,37.0,37.0,32.0,21.0,7.0,58.0,0.0,1019.0,32.0,10.0,95.0,18.0,73.0


In [5]:
start_time = raw_data.index.min()
end_time = raw_data.index.max()
complete_range = pd.date_range(start=start_time, end=end_time, freq='h')

# Find missing hours and then insert these times to make the time consecutive
missing_hours = complete_range.difference(raw_data.index)
print('missing hours before: ', len(missing_hours))
missing_df = pd.DataFrame(index=missing_hours)
raw_data = pd.concat([raw_data, missing_df])
raw_data = raw_data.sort_index()

filled_df = strategic_fill(raw_data)
missing_hours = complete_range.difference(filled_df.index)
print('missing hours after: ', len(missing_hours))

missing hours before:  65
missing hours after:  0


In [6]:
filled_df.head()

,Homestead_maxtempC,Homestead_mintempC,Homestead_DewPointC,Homestead_FeelsLikeC,Homestead_HeatIndexC,Homestead_WindChillC,Homestead_WindGustKmph,Homestead_cloudcover,Homestead_humidity,Homestead_precipMM,Homestead_pressure,Homestead_tempC,Homestead_visibility,Homestead_winddirDegree,Homestead_windspeedKmph,Consumption
2017-12-11 00:00:00+00:00,19.0,9.0,1.0,8.0,10.0,8.0,28.0,0.0,55.0,0.0,1024.0,10.0,10.0,335.0,16.0,56.0
2017-12-11 01:00:00+00:00,19.0,9.0,2.0,8.0,10.0,8.0,28.0,0.0,55.0,0.0,1024.0,10.0,10.0,337.0,16.0,56.0
2017-12-11 02:00:00+00:00,19.0,9.0,2.0,8.0,10.0,8.0,28.0,0.0,56.0,0.0,1024.0,10.0,10.0,338.0,16.0,55.0
2017-12-11 03:00:00+00:00,19.0,9.0,2.0,8.0,10.0,8.0,29.0,0.0,57.0,0.0,1024.0,10.0,10.0,339.0,16.0,52.0
2017-12-11 04:00:00+00:00,19.0,9.0,1.0,7.0,10.0,7.0,29.0,0.0,54.0,0.0,1024.0,10.0,10.0,342.0,17.0,46.0


In [7]:
# Check the current index name
print(f"Current index name: {filled_df.index.name}")

# Reset the index to make it a column, then rename it to 'time'
filled_df = filled_df.reset_index()
filled_df.rename(columns={'index': 'time'}, inplace=True)

# Set 'time' as the index again
filled_df.set_index('time', inplace=True)

# Verify the change
print(f"New index name: {filled_df.index.name}")


Current index name: None
New index name: time


In [8]:
filled_df.head()

,Homestead_maxtempC,Homestead_mintempC,Homestead_DewPointC,Homestead_FeelsLikeC,Homestead_HeatIndexC,Homestead_WindChillC,Homestead_WindGustKmph,Homestead_cloudcover,Homestead_humidity,Homestead_precipMM,Homestead_pressure,Homestead_tempC,Homestead_visibility,Homestead_winddirDegree,Homestead_windspeedKmph,Consumption
time,,,,,,,,,,,,,,,,
2017-12-11 00:00:00+00:00,19.0,9.0,1.0,8.0,10.0,8.0,28.0,0.0,55.0,0.0,1024.0,10.0,10.0,335.0,16.0,56.0
2017-12-11 01:00:00+00:00,19.0,9.0,2.0,8.0,10.0,8.0,28.0,0.0,55.0,0.0,1024.0,10.0,10.0,337.0,16.0,56.0
2017-12-11 02:00:00+00:00,19.0,9.0,2.0,8.0,10.0,8.0,28.0,0.0,56.0,0.0,1024.0,10.0,10.0,338.0,16.0,55.0
2017-12-11 03:00:00+00:00,19.0,9.0,2.0,8.0,10.0,8.0,29.0,0.0,57.0,0.0,1024.0,10.0,10.0,339.0,16.0,52.0
2017-12-11 04:00:00+00:00,19.0,9.0,1.0,7.0,10.0,7.0,29.0,0.0,54.0,0.0,1024.0,10.0,10.0,342.0,17.0,46.0


In [9]:
print(f"Total null values: {filled_df.isna().sum().sum()}")
print("\nNull values by column:")
print(filled_df.isna().sum())

if filled_df.isna().any().any():
    print("\nPercentage of null values in each column:")
    null_percentage = (filled_df.isna().sum() / len(filled_df)) * 100
    print(null_percentage[null_percentage > 0].sort_values(ascending=False))


Total null values: 0

Null values by column:
Homestead_maxtempC         0
Homestead_mintempC         0
Homestead_DewPointC        0
Homestead_FeelsLikeC       0
Homestead_HeatIndexC       0
Homestead_WindChillC       0
Homestead_WindGustKmph     0
Homestead_cloudcover       0
Homestead_humidity         0
Homestead_precipMM         0
Homestead_pressure         0
Homestead_tempC            0
Homestead_visibility       0
Homestead_winddirDegree    0
Homestead_windspeedKmph    0
Consumption                0
dtype: int64


In [10]:
# Hour of the Day (0 to 23)
filled_df['HoD'] = filled_df.index.hour
# Day of the Week (1=Monday to 7=Sunday)
filled_df['DoW'] = filled_df.index.dayofweek + 1
filled_df['MoY'] = filled_df.index.month

In [11]:
filled_df[['HoD', 'DoW', 'MoY']].head()

,HoD,DoW,MoY
time,,,
2017-12-11 00:00:00+00:00,0,1,12
2017-12-11 01:00:00+00:00,1,1,12
2017-12-11 02:00:00+00:00,2,1,12
2017-12-11 03:00:00+00:00,3,1,12
2017-12-11 04:00:00+00:00,4,1,12


In [12]:
filled_df.to_csv(f'{project_root}/data/cleaned/cleaned_us_homestead.csv')

In [13]:
filled_df.columns

Index(['Homestead_maxtempC', 'Homestead_mintempC', 'Homestead_DewPointC',
       'Homestead_FeelsLikeC', 'Homestead_HeatIndexC', 'Homestead_WindChillC',
       'Homestead_WindGustKmph', 'Homestead_cloudcover', 'Homestead_humidity',
       'Homestead_precipMM', 'Homestead_pressure', 'Homestead_tempC',
       'Homestead_visibility', 'Homestead_winddirDegree',
       'Homestead_windspeedKmph', 'Consumption', 'HoD', 'DoW', 'MoY'],
      dtype='object')

In [14]:
len(list(filled_df.columns))

19

In [15]:
len(filled_df)

22266

In [16]:
import math

tr_s, tr_e = 0, math.ceil(0.6 * 22266)
v_s, v_e = tr_e + 1, tr_e + 1 + math.floor(0.2 * 22266)
te_s, te_e = v_e + 1, v_e + math.floor(0.2 * 22266)

In [19]:
te_e

22267

In [21]:
# Get the dates at specific indices to determine train/val/test splits
dates = filled_df.index.tolist()

# Format dates for config JSON files
train_start = dates[tr_s].strftime("%Y-%m-%d %H:%M:%S%z")
train_end = dates[tr_e].strftime("%Y-%m-%d %H:%M:%S%z")
val_start = dates[v_s].strftime("%Y-%m-%d %H:%M:%S%z")
val_end = dates[v_e].strftime("%Y-%m-%d %H:%M:%S%z")
test_start = dates[te_s].strftime("%Y-%m-%d %H:%M:%S%z")
test_end = dates[te_e - 2].strftime("%Y-%m-%d %H:%M:%S%z")

print(f'"train_start": "{train_start}",')
print(f'"train_end": "{train_end}",')
print(f'"val_start": "{val_start}",')
print(f'"val_end": "{val_end}",')
print(f'"test_start": "{test_start}",')
print(f'"test_end": "{test_end}"')

# Calculate the number of samples in each split
train_samples = tr_e - tr_s + 1
val_samples = v_e - v_s + 1
test_samples = te_e - te_s + 1

print(f"\nNumber of training samples: {train_samples}")
print(f"Number of validation samples: {val_samples}")
print(f"Number of testing samples: {test_samples}")


"train_start": "2017-12-11 00:00:00+0000",
"train_end": "2019-06-20 16:00:00+0000",
"val_start": "2019-06-20 17:00:00+0000",
"val_end": "2019-12-23 06:00:00+0000",
"test_start": "2019-12-23 07:00:00+0000",
"test_end": "2020-06-25 17:00:00+0000"

Number of training samples: 13361
Number of validation samples: 4454
Number of testing samples: 4453
